# 01 - Exploratory Data Analysis: Crashes Involving Fatalities

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt     # data visualization library
import seaborn as sns     # prettier data visualization library than matplotlib
from io import BytesIO     # From https://www.geeksforgeeks.org/stringio-and-bytesio-for-managing-data-as-file-object/#
import requests     # simple https library for python https://www.geeksforgeeks.org/python-requests-tutorial/
from datetime import datetime     # https://www.geeksforgeeks.org/python-datetime-module/
import re     # RegEx or Regular Expression library

In [4]:
# Set the style for plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("deep")

In [16]:
# Load the data of fatal crashes.
df = pd.read_csv('data/Traffic_Crashes_Resulting_in_Fatality_20250411.csv')
df.head()

,unique_id,case_id_fkey,latitude,longitude,collision_year,death_date,death_time,death_datetime,collision_date,collision_time,...,in_coc_2018,publish,on_vz_hin_2022,in_epa_2021,point,analysis_neighborhood,supervisor_district,police_district,data_as_of,data_loaded_at
0,1,140236301,37.710409,-122.404226,2014,2014/03/20,11:21:00,2014/03/20 11:21:00 AM,2014/03/20,NaN,...,False,True,True,False,POINT (-122.404226037 37.710409217),Bayview Hunters Point,10.0,INGLESIDE,2024/09/18 12:00:00 AM,2025/04/08 04:35:01 PM
1,2,140755533,37.725476,-122.394243,2014,2014/09/08,16:38:00,2014/09/08 04:38:00 PM,2014/09/08,05:10:00,...,True,True,True,True,POINT (-122.394243493 37.72547565),Bayview Hunters Point,10.0,BAYVIEW,2024/09/18 12:00:00 AM,2025/04/08 04:35:01 PM
2,4,140365546,37.748255,-122.413669,2014,2014/05/03,17:20:00,2014/05/03 05:20:00 PM,2014/05/03,02:24:00,...,False,True,True,False,POINT (-122.413668844 37.748255329),Mission,9.0,MISSION,2024/09/18 12:00:00 AM,2025/04/08 04:35:01 PM
3,16,150562049,37.777300,-122.419694,2015,2015/06/30,06:00:00,2015/06/30 06:00:00 AM,2015/06/28,03:52:00,...,False,True,True,True,POINT (-122.419693566 37.777299856),Tenderloin,5.0,NORTHERN,2024/09/18 12:00:00 AM,2025/04/08 04:35:01 PM
4,17,140104811,37.778251,-122.419883,2014,2014/02/06,10:20:00,2014/02/06 10:20:00 AM,2014/02/05,02:26:00,...,False,True,True,True,POINT (-122.419883231 37.778251017),Hayes Valley,5.0,NORTHERN,2024/09/18 12:00:00 AM,2025/04/08 04:35:01 PM


In [10]:
# 1. Analyze the dataset structure
print("Dataset Shape:", df.shape)
print("\nData Types:")
print(df.dtypes)

Dataset Shape: (331, 28)

Data Types:
unique_id                  int64
case_id_fkey              object
latitude                 float64
longitude                float64
collision_year             int64
death_date                object
death_time                object
death_datetime            object
collision_date            object
collision_time            object
collision_datetime        object
location                  object
age                      float64
sex                       object
deceased                  object
collision_type            object
street_type               object
on_vz_hin_2017              bool
in_coc_2018                 bool
publish                     bool
on_vz_hin_2022              bool
in_epa_2021                 bool
point                     object
analysis_neighborhood     object
supervisor_district      float64
police_district           object
data_as_of                object
data_loaded_at            object
dtype: object


In [11]:
# Check for missing values
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percent
})
print("\nMissing Values Analysis:")
print(missing_data[missing_data['Missing Values'] > 0].sort_values('Missing Values', ascending=False))


Missing Values Analysis:
                       Missing Values  Percentage
death_time                         38   11.480363
collision_time                     25    7.552870
case_id_fkey                        7    2.114804
age                                 1    0.302115
analysis_neighborhood               1    0.302115
supervisor_district                 1    0.302115


In [12]:
# Check for duplicates
duplicate_rows = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_rows}")

# Check for inconsistencies in date/time formats
print("\nDate/Time Format Examples:")
for col in ['death_date', 'death_time', 'death_datetime', 'collision_date', 'collision_time', 'collision_datetime']:
    print(f"{col}: {df[col].dropna().iloc[0]}")


Number of duplicate rows: 0

Date/Time Format Examples:
death_date: 2014/03/20
death_time: 11:21:00
death_datetime: 2014/03/20 11:21:00 AM
collision_date: 2014/03/20
collision_time: 05:10:00
collision_datetime: 2014/03/20 12:00:00 AM


In [15]:
# 2. Clean the data
# Make a copy of the original dataframe for comparison
df_original = df.copy()

# Function to safely convert date strings to datetime
def safe_parse_date(date_str, format_str):
    if pd.isna(date_str):
        return pd.NaT
    try:
        return pd.to_datetime(date_str, format=format_str)
    except:
        return pd.NaT

# Function to safely convert time strings to time objects
def safe_parse_time(time_str, format_str):
    if pd.isna(time_str):
        return pd.NaT
    try:
        return pd.to_datetime(time_str, format=format_str).time()
    except:
        return pd.NaT


# 1. Clean date columns with explicit formats
df['death_date'] = df['death_date'].apply(lambda x: safe_parse_date(x, '%Y/%m/%d'))
df['collision_date'] = df['collision_date'].apply(lambda x: safe_parse_date(x, '%Y/%m/%d'))
df['data_as_of'] = df['data_as_of'].apply(lambda x: safe_parse_date(x, '%Y/%m/%d %H:%M:%S %p'))
df['data_loaded_at'] = df['data_loaded_at'].apply(lambda x: safe_parse_date(x, '%Y/%m/%d %H:%M:%S %p'))

# 2. Clean time columns
# First convert to datetime, then extract time component
for col in ['death_time', 'collision_time']:
    df[col] = df[col].apply(lambda x: safe_parse_time(x, '%H:%M:%S') if pd.notna(x) else pd.NaT)

# 3. Clean datetime columns with explicit formats
for col in ['death_datetime', 'collision_datetime']:
    df[col] = df[col].apply(lambda x: safe_parse_date(x, '%Y/%m/%d %H:%M:%S %p'))

# 4. Create age categories for better analysis
age_bins = [0, 18, 35, 65, 100]
age_labels = ['Child (0-17)', 'Young Adult (18-34)', 'Adult (35-64)', 'Senior (65+)']
df['age_category'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=False)

# 5. Create a more meaningful collision type category
# Simplify collision types into broader categories
def simplify_collision_type(collision_type):
    if pd.isna(collision_type):
        return 'Unknown'
    elif 'Pedestrian' in collision_type:
        return 'Pedestrian'
    elif 'Bicycle' in collision_type:
        return 'Bicycle'
    elif 'Motorcycle' in collision_type:
        return 'Motorcycle'
    else:
        return 'Vehicle'

df['collision_category'] = df['collision_type'].apply(simplify_collision_type)

# 6. Fix data type issues
# Convert boolean columns to proper boolean type
bool_columns = ['on_vz_hin_2017', 'in_coc_2018', 'publish', 'on_vz_hin_2022', 'in_epa_2021']
for col in bool_columns:
    df[col] = df[col].astype(bool)

# Ensure numeric columns are properly typed
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['supervisor_district'] = pd.to_numeric(df['supervisor_district'], errors='coerce')
df['collision_year'] = pd.to_numeric(df['collision_year'], errors='coerce')

# 7. Extract hour of day from collision_datetime for time analysis
df['collision_hour'] = df['collision_datetime'].apply(
    lambda x: x.hour if pd.notna(x) else np.nan
)

# 8. Create a time of day category
def categorize_time_of_day(hour):
    if pd.isna(hour):
        return 'Unknown'
    elif 5 <= hour < 12:
        return 'Morning (5am-12pm)'
    elif 12 <= hour < 17:
        return 'Afternoon (12pm-5pm)'
    elif 17 <= hour < 21:
        return 'Evening (5pm-9pm)'
    else:
        return 'Night (9pm-5am)'

df['time_of_day'] = df['collision_hour'].apply(categorize_time_of_day)

# Summarize the changes made during cleaning
changes = {
    'Date format fixes': sum(pd.notna(df['death_date']) & pd.isna(df_original['death_date'].apply(lambda x: pd.to_datetime(x, errors='coerce')))),
    'Time parsing fixes': sum(pd.notna(df['collision_time']) & pd.isna(df_original['collision_time'].apply(lambda x: pd.to_datetime(x, format='%H:%M:%S', errors='coerce')))),
    'Age categories created': df['age_category'].notna().sum(),
    'Collision categories created': df['collision_category'].notna().sum(),
    'Time of day categories created': df['time_of_day'].notna().sum()
}

# Handle missing values
# For numeric columns, fill with median
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# For categorical columns, fill with mode
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_clean[col].isnull().sum() > 0 and col not in ['collision_time', 'death_time']:
        df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

# Remove duplicates
df_clean = df_clean.drop_duplicates()

# Convert date/time columns to proper datetime format
date_cols = ['death_date', 'collision_date']
datetime_cols = ['death_datetime', 'collision_datetime', 'data_as_of', 'data_loaded_at']

for col in date_cols:
    df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

for col in datetime_cols:
    df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Extract year, month, day, hour from datetime columns for analysis
df_clean['collision_year_clean'] = df_clean['collision_year']
df_clean['collision_month'] = df_clean['collision_date'].dt.month
df_clean['collision_day'] = df_clean['collision_date'].dt.day
df_clean['collision_hour'] = pd.to_datetime(df_clean['collision_time'], format='%H:%M:%S', errors='coerce').dt.hour

# Standardize formats for categorical variables
df_clean['deceased'] = df_clean['deceased'].str.strip()
df_clean['collision_type'] = df_clean['collision_type'].str.strip()

print("Summary of Data Cleaning Changes:")
for change, count in changes.items():
    print(f"- {change}: {count} rows affected")

# Print basic statistics of the cleaned dataset
print("\nCleaned Dataset Statistics:")
print(f"Total records: {len(df)}")
print(f"Date range: {df['collision_date'].min()} to {df['collision_date'].max()}")
print(f"Age range: {df['age'].min()} to {df['age'].max()}, Mean: {df['age'].mean():.1f}")

Summary of Data Cleaning Changes:
- Date format fixes: 0 rows affected
- Time parsing fixes: 0 rows affected
- Age categories created: 330 rows affected
- Collision categories created: 331 rows affected
- Time of day categories created: 331 rows affected

Cleaned Dataset Statistics:
Total records: 331
Date range: 2014-01-04 00:00:00 to 2025-02-07 00:00:00
Age range: 0.0 to 95.0, Mean: 52.1


In [18]:
df.to_csv('./data/cleaned_fatalities.csv', index=False)